# Téléchargement des PDF sur Arkiv

In [1]:
import arxiv
import urllib
import pdfx

In [2]:
#Utilisation de la librairie ArXiv pou récupérer le lien des pdf

search = arxiv.Search(
    query = "computer science &  ai",
    max_results = 1,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
    print ("Titre :", result.title)
    print ("Auteurs :", result.authors)
    print ("Lien :", result.pdf_url)

Titre : Simplicial Embeddings in Self-Supervised Learning and Downstream Classification
Auteurs : [arxiv.Result.Author('Samuel Lavoie'), arxiv.Result.Author('Christos Tsirigotis'), arxiv.Result.Author('Max Schwarzer'), arxiv.Result.Author('Kenji Kawaguchi'), arxiv.Result.Author('Ankit Vani'), arxiv.Result.Author('Aaron Courville')]
Lien : http://arxiv.org/pdf/2204.00616v1


In [3]:
#Download the pdf & store it

response = urllib.request.urlopen(result.pdf_url)    
file = open("Test" + ".pdf", 'wb')
file.write(response.read())
file.close()

In [4]:
pdf = pdfx.PDFx("Test.pdf")
metadata = pdf.get_metadata()
references_list = pdf.get_references()
references_dict = pdf.get_references_as_dict()
text = pdf.get_text()

In [8]:

print (metadata)
#print (references_list)
#print (references_dict)
#print (text)

{'CreationDate': 'D:20220404004450Z', 'Creator': 'LaTeX with hyperref', 'ModDate': 'D:20220404004450Z', 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'Producer': 'pdfTeX-1.40.21', 'Trapped': 'False', 'Pages': 22}


## Optimisation du champ de recherche à partir du mot "Références"

In [9]:
def after_references(mypdftext): 
    keyword1 = 'References'
    keyword2 = 'REFERENCES'
    keyword3 = 'R EFERENCES'
    keyword4 = 'Reference'
    keyword5='[1]' 

    if keyword1 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword1)
    elif keyword2 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword2)
    elif keyword3 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword3)
    elif keyword4 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword4)
    elif keyword5 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword5)
    else:
        after_keyword = mypdftext[:10000]
    return after_keyword

#All references in a variable

references=after_references(text)
print(references)



Martin Arjovsky, Léon Bottou, Ishaan Gulrajani, and David Lopez-Paz. Invariant Risk Minimization.
arXiv:1907.02893 [cs, stat], March 2020. URL http://arxiv.org/abs/1907.02893. arXiv:
1907.02893.

Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural Machine Translation by Jointly
Learning to Align and Translate. arXiv:1409.0473 [cs, stat], May 2016. URL http://arxiv.
org/abs/1409.0473. arXiv: 1409.0473.

Lukas Bossard, Matthieu Guillaumin, and Luc Van Gool. Food-101 – mining discriminative com-
ponents with random forests. In David Fleet, Tomas Pajdla, Bernt Schiele, and Tinne Tuytelaars,
editors, Computer Vision – ECCV 2014, pages 446–461, Cham, 2014. Springer International
Publishing. ISBN 978-3-319-10599-4.

Léon Bottou, Frank E. Curtis, and Jorge Nocedal. Optimization Methods for Large-Scale Machine
Learning. arXiv:1606.04838 [cs, math, stat], February 2018. URL http://arxiv.org/abs/
1606.04838. arXiv: 1606.04838.

Xavier Bouthillier, Christos Tsirigotis, François Corneau-Tr

# Recherche des entités nommées

In [20]:
import spacy
from spacy.lang.fr.examples import sentences 

def extract(text:str) :
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text.strip())
    named_entities = []
    
    for i in doc.ents:
        entry = str(i.lemma_).lower()
        text = text.replace(str(i).lower(), "")
        if i.label_ in ["PERSON"]:
            named_entities.append(entry.title().replace(" ", "_").replace("\n","_"))
        named_entities = list(dict.fromkeys(named_entities))
    return named_entities

print (extract(references)[4])

Kyunghyun_Cho
